# 1 Time Series 7-Day Forecasting with multi-layer perceptron (MLP)<a id='1_time_series_7-day_forecasting_with_mlp'></a>

## 1.1 Contents<a id='1.1_contents'></a>
* [1 Time series 7-day forecasting with MLP](#1_time_series_7-day_forecasting_with_mlp)
  * [1.1 Contents](#1.1_contents)  
  * [1.2 Imports](#1.2_imports)
  * [1.3 Functions](#1.3_functions)
      * [1.3.1 Function: feature_list](#1.3.1_feature_list)
      * [1.3.2 Function: ebd_dim](#1.3.2_ebd_dim)
      * [1.3.3 Function: split_sequences](#1.3.3_split_sequences)
      * [1.3.4 Function: to_embed](#1.3.4_to_embed)
      * [1.3.5 Function: build_embedding_network](#1.3.5_build_embedding_network)
      * [1.3.6 Function: ndarray_to_input_list](#1.3.6_ndarray_to_input_list)
      * [1.3.7 Function: ginic](#1.3.7_ginic)
      * [1.3.8 Function: gini_normalizedc](#1.3.8_gini_normalizedc)

## 1.2 Imports<a id='1.2_imports'></a>

In [446]:
import pandas as pd
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
import pickle
from pathlib import Path
from datetime import datetime, timedelta
from isoweek import Week
import math

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import TimeSeriesSplit

from keras.models import Model
from keras.layers import Input, Dense, Concatenate, Reshape, Dropout
from keras.layers.embeddings import Embedding
from keras.utils.vis_utils import plot_model

## 1.3 Functions<a id='1.3_functions'></a>

#### 1.3.1 Functions: feature_list <a id='1.3.1_feature_list'></a>

In [480]:
# organize features in each row into 1) static categorical, 2) temporal categorical, 3) temporal continuous
def feature_list(country_id, row): 
    # Static Categorical
    country = country_id #0 country id
    # Temporal Categorical (datetime variables)
    dt = row[0].to_pydatetime()
    year = dt.year #1
    month = dt.month #2
    day = dt.day #3
    week_of_year = dt.isocalendar()[1] #4
    day_of_week = row[1].dayow #5
    holiday = row[1].holiday #6 holiday
    # Temporal Continuous (mobility variables)
    rtrc = row[1].rtrc #7 retail & recreation
    grph = row[1].grph #8 grocery & pharmacy
    prks = row[1].prks #9 parks
    tran = row[1].tran #10 transportation
    work = row[1].work #11 workplace
    resi = row[1].resi #12 residential area
    # Temporal Continuous (weather variables)
    cloudcover = float(row[1].cloudcover) #13 weather; cloudcover
    tempC = float(row[1].tempC) #14 weather; temparature
    humidity = float(row[1].humidity) #15 weather; humidity
    precipMM = float(row[1].precipMM) #16 weather; precipitation
    # Temporal Continuous (vaccination)
    vac = row[1].vac #17 vaccination
            
    return [country], \
[year, month, day, week_of_year, day_of_week, holiday], \
[rtrc, grph, prks, tran, work, resi, cloudcover, tempC, humidity, precipMM, vac] # Static Categorical, Temporal Categorical, Temporal Continuous

#### 1.3.2 Functions: ebd_dim <a id='1.3.2_ebd_dim'></a>

In [449]:
# determine the embedding dimensions
def ebd_dim(cat_dim):
    return min(600, round(1.6 * cat_dim ** .56))

#### 1.3.3 Functions: split_sequences <a id='1.3.3_split_sequences'></a>

In [450]:
# get the input and output sequences from the entire time series
def split_sequences(sequences, timestamp, n_steps_in, n_steps_out): 
    timestamps = sequences.index
    df_time0 = timestamps[0]
    df_time_end = timestamps[-1]
    dt_steps_in = timedelta(days=n_steps_in)
    dt_steps_out = timedelta(days=n_steps_out-1)   
    dt_1 = timedelta(days=1)
    if (timestamp-dt_steps_in>=df_time0) & (timestamp+dt_steps_out<=df_time_end): # if within bounds
        # gather input and output parts of the pattern 
        seq_x = sequences[timestamp-dt_steps_in:timestamp-dt_1] # input sequence (e.g. previous 14 days) 
        seq_y = sequences[timestamp:timestamp+dt_steps_out] # output sequence (e.g. next 7 days including the current timestamp)
    return list(seq_x), list(seq_y)

#### 1.3.4 Functions: to_embed <a id='1.3.4_to_embed'></a>

In [451]:
# preprocess the embedding columns
def to_embed(cat_columns): 
    
    cat_conv_raw = []
    cat_conv_array = np.empty((cat_columns.shape[0],cat_columns.shape[1]))
    
    for c in range(cat_columns.shape[1]):
        cat_conv_raw.append(list(cat_columns[:,c]))
        raw_vals = np.unique(cat_columns[:,c])
        val_map = {}
        for i in range(len(raw_vals)):
            val_map[raw_vals[i]] = i
        cat_conv_array[:,c] = [val_map[j] for j in cat_conv_raw[c]]
    return cat_conv_array

#### 1.3.5 Functions: build_embedding_network <a id='1.3.5_build_embedding_network'></a>

In [452]:
def build_embedding_network(): 
    
    inputs = []
    embeddings = []
    input_dims = []
    
    input_stat_cat_00_country = Input(shape=(1,)) # country (0)
    embedding = Embedding(23, 9, input_length=1)(input_stat_cat_00_country)
    embedding = Reshape(target_shape=(9,))(embedding)
    inputs.append(input_stat_cat_00_country)
    embeddings.append(embedding)
    input_dims.append(input_stat_cat_00_country.shape[-1])
    
    input_temp_cat_01_year = Input(shape=(1,)) # year (1)
    embedding = Embedding(2, 1, input_length=1)(input_temp_cat_01_year)
    embedding = Reshape(target_shape=(1,))(embedding)
    inputs.append(input_temp_cat_01_year)
    embeddings.append(embedding)
    input_dims.append(input_temp_cat_01_year.shape[-1])
     
    input_temp_cat_02_month = Input(shape=(1,)) # month (2)
    embedding = Embedding(12, 6, input_length=1)(input_temp_cat_02_month)
    embedding = Reshape(target_shape=(6,))(embedding)
    inputs.append(input_temp_cat_02_month)
    embeddings.append(embedding)    
    input_dims.append(input_temp_cat_02_month.shape[-1])
    
    input_temp_cat_03_day = Input(shape=(1,)) # day (3)
    embedding = Embedding(31, 11, input_length=1)(input_temp_cat_03_day)
    embedding = Reshape(target_shape=(11,))(embedding)
    inputs.append(input_temp_cat_03_day)
    embeddings.append(embedding)    
    input_dims.append(input_temp_cat_03_day.shape[-1])
    
    input_temp_cat_04_week_of_year = Input(shape=(1,)) # week of year (4)
    embedding = Embedding(53, 15, input_length=1)(input_temp_cat_04_week_of_year)
    embedding = Reshape(target_shape=(15,))(embedding)
    inputs.append(input_temp_cat_04_week_of_year)
    embeddings.append(embedding) 
    input_dims.append(input_temp_cat_04_week_of_year.shape[-1])
    
    input_temp_cat_05_day_of_week = Input(shape=(1,)) # day of week (5)
    embedding = Embedding(7, 5, input_length=1)(input_temp_cat_05_day_of_week)
    embedding = Reshape(target_shape=(5,))(embedding)
    inputs.append(input_temp_cat_05_day_of_week)
    embeddings.append(embedding) 
    input_dims.append(input_temp_cat_05_day_of_week.shape[-1])
    
    input_temp_cat_06_holiday = Input(shape=(1,)) # holiday (6)
    embedding = Embedding(2, 2, input_length=1)(input_temp_cat_06_holiday)
    embedding = Reshape(target_shape=(2,))(embedding)
    inputs.append(input_temp_cat_06_holiday)
    embeddings.append(embedding)  
    input_dims.append(input_temp_cat_06_holiday.shape[-1])
    
    input_numeric = Input(shape=(25,)) # all temporal continuous features (mobility, weather, vaccination)
    dense_numeric = Dense(19)(input_numeric)
    inputs.append(input_numeric)
    embeddings.append(dense_numeric)
    input_dims.append(input_numeric.shape[-1])
    
    x = Concatenate()(embeddings)
    x = Dense(90, activation='relu')(x)
    x = Dropout(.35)(x)
    x = Dense(40, activation='relu')(x)
    x = Dropout(.15)(x)
    x = Dense(10, activation='relu')(x)
    x = Dropout(.15)(x)
    output = Dense(7, activation='linear')(x)
    
    model = Model(inputs, output)
    
    model.compile(loss='binary_crossentropy', optimizer='adam')
    
    return model, input_dims
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
# print(model.summary())

#### 1.3.6 Functions: ndarray_to_input_list <a id='1.3.6_ndarray_to_input_list'></a>

In [453]:
def ndarray_to_input_list(ndarr,input_dims): 
    input_list = []
    input_dim_cumsum = list(np.cumsum(input_dims))
    input_dim_cumsum.insert(0,0)
    curr_input_0 = input_dim_cumsum[:-1]
    for i,input_0 in enumerate(curr_input_0):
        input_list.append(ndarr[:,input_0:input_0+input_dims[i]])
    return input_list

#### 1.3.7 Functions: ginic <a id='1.3.7_ginic'></a>

In [454]:
#gini scoring function from kernel at: 
#https://www.kaggle.com/tezdhar/faster-gini-calculation
def ginic(actual, pred):
    pred_arr = np.asarray(pred)
    if len(pred_arr.shape)==2: # matrix 
        pred_mean = pred.copy()
    elif len(pred_arr.shape)==3: # tuple 
        pred_mean = np.nanmean(pred_arr,axis=0) # average across cross-validation folds
    n, c = actual.shape[0], actual.shape[1]
    gini_sum = []
    for col in range(c):
        a_s = actual[np.argsort(pred_mean[:,col]),col] # returns the indices that would sort an array
        a_c = a_s.cumsum()
        gini_sum.append((a_c.sum() / a_c[-1] - (n + 1) / 2.0)/n) 
    return gini_sum

#### 1.3.8 Functions: gini_normalizedc <a id='1.3.8_gini_normalizedc'></a>

In [455]:
def gini_normalizedc(a, p):
    gini_a_p = ginic(a, p)
    gini_a_a = ginic(a, a)
    gini_norm = [gini_a_p[i]/gini_a_a[i] for i in range(len(gini_a_p))]
    return gini_norm

In [447]:
# load the saved dictionary from pickle file
filePath_pickle = Path('/Users/parkj/Documents/pyDat/dataSet/covid_country_data.pickle')
with open(filePath_pickle, 'rb') as f:
    dict_country = pickle.load(f)
# countries = ['AR', 'AT', 'AU', 'BE', 'CA', 'DE', 'DK', 'FI', 'FR', 'GB', 'ID', 'IE', 'IL', 'IN', 'IT', 'JP', 'KR', 'MX', 'NL', 'NO', 'RU', 'SG', 'US']

#### Get features and target for train and test sets

In [456]:
train_timestamp = []
train_stat_cat = []
train_temp_cat = []
train_temp_con = []
train_feature_case = []
train_y_unscaled = []
test_timestamp = []
test_stat_cat = []
test_temp_cat = []
test_temp_con = []
test_feature_case = []
test_y_unscaled = []

n_test = 50 # days
dt_test = timedelta(days=n_test-1)
n_steps_in = 14 # days (# previous cases)
dt_steps_in = timedelta(days=n_steps_in)
n_steps_out = 7 # days (# future cases to be predicted)
dt_steps_out = timedelta(days=n_steps_out-1)

case_detection = 0

for i, country_key in enumerate(dict_country.keys()):
    df_country = dict_country[country_key]
    df_country.fillna(method='ffill',inplace=True) # forward fill NaNs
    df_time0 = df_country.index[0] # the first day of the data
    df_time_end = df_country.index[-1] # the last day of the data
    # split the df into train and test sets
    test_time0 = df_country.index[-1]-dt_test # the first date of test set 
    train_ind = df_country.index < test_time0 # training index
    # feature_list train 
    df_country_train = df_country.loc[train_ind] # train df
    for row in df_country_train.iterrows():
        ts_curr = row[0] 
        # case_mil lagging
        if (ts_curr-dt_steps_in>=df_time0) & (ts_curr+dt_steps_out<=df_time_end):
            # get feature and target variables
            feature_case, target_case = split_sequences(df_country['case_mil'], ts_curr, n_steps_in, n_steps_out)         
            if (case_detection == 0) & (sum(feature_case)>0):
                case_detection = 1
            if case_detection == 1:
                fl_stat_cat, fl_temp_cat, fl_temp_con = feature_list(i, row) # get static categorical, temporal categorical, temporal continuous variables separately 
                # train data X
                train_timestamp.append(ts_curr) # timestamps
                train_stat_cat.append(fl_stat_cat) # static categorical 
                train_temp_cat.append(fl_temp_cat) # temporal categorical 
                train_temp_con.append(fl_temp_con) # temporal continuous
                train_feature_case.append(feature_case) # case_mil previous days to be used as features
                # train data y
                train_y_unscaled.append(target_case) # case_mil current & future days to be predicted
    # feature list test
    df_country_test = df_country.loc[~train_ind] # test df
    # feature list test 
    for row in df_country_test.iterrows():
        ts_curr = row[0]
        # case_mil lagging
        if (ts_curr-dt_steps_in>=df_time0) & (ts_curr+dt_steps_out<=df_time_end):        
            # get feature and target variables
            feature_case, target_case = split_sequences(df_country['case_mil'], ts_curr, n_steps_in, n_steps_out) 
            fl_stat_cat, fl_temp_cat, fl_temp_con = feature_list(i, row) # get static categorical, temporal categorical, temporal continuous variables separately
            # test data X
            test_timestamp.append(ts_curr)
            test_stat_cat.append(fl_stat_cat) # static categorical 
            test_temp_cat.append(fl_temp_cat) # temporal categorical 
            test_temp_con.append(fl_temp_con) # temporal continuous
            test_feature_case.append(feature_case) # case_mil previous days to be used as features
            # train data y
            test_y_unscaled.append(target_case) # case_mil current & future days to be predicted

#### Scale features and target

In [457]:
# convert list to ndarray
train_stat_cat_embed = to_embed(np.array(train_stat_cat))
train_temp_cat_embed = to_embed(np.array(train_temp_cat))
train_temp_con = np.array(train_temp_con)
train_feature_case = np.reshape(train_feature_case, (len(train_timestamp), n_steps_in))
train_y_unscaled = np.reshape(train_y_unscaled, (len(train_timestamp), n_steps_out))
test_stat_cat_embed = to_embed(np.array(test_stat_cat))
test_temp_cat_embed = to_embed(np.array(test_temp_cat))
test_temp_con = np.array(test_temp_con)
test_feature_case = np.reshape(test_feature_case, (len(test_timestamp), n_steps_in))
test_y_unscaled = np.reshape(test_y_unscaled, (len(test_timestamp), n_steps_out))
# scale the temporal continuous variables
temp_con_scaler = MinMaxScaler()
temp_con_scaler.fit(train_temp_con)
train_temp_con_scaled = temp_con_scaler.transform(train_temp_con)
test_temp_con_scaled = temp_con_scaler.transform(test_temp_con)
# scale feature and target cases (case_mil)
scaler_feature_case = StandardScaler()
scaler_y = StandardScaler()
case_all_1d = np.concatenate((train_feature_case.reshape(-1,1), test_feature_case.reshape(-1,1),\
                            train_y_unscaled.reshape(-1,1), test_y_unscaled.reshape(-1,1)),axis=0)
y_all_feature = np.matlib.repmat(case_all_1d, 1, train_feature_case.shape[1])
y_all_test = np.matlib.repmat(case_all_1d, 1, train_y_unscaled.shape[1])

scaler_feature_case.fit(y_all_feature)
scaler_y.fit(y_all_test)

train_feature_case_scaled = scaler_feature_case.transform(train_feature_case)
test_feature_case_scaled = scaler_feature_case.transform(test_feature_case)

train_y = scaler_y.transform(train_y_unscaled)
test_y = scaler_y.transform(test_y_unscaled)
# concatenate
train_X = np.concatenate((train_stat_cat_embed, train_temp_cat_embed, train_temp_con_scaled, train_feature_case_scaled), axis=1)
test_X = np.concatenate((test_stat_cat_embed, test_temp_cat_embed, test_temp_con_scaled, test_feature_case_scaled), axis=1)
print("Number of train datapoints: ", len(train_y))
print("Number of test datapoints: ", len(test_y))

Number of train datapoints:  11887
Number of test datapoints:  1012


In [466]:
# get the cross-validation folds for train and validation sets  
train_folds = []
validation_folds = []
# get the timestamps for train and validation folds
sorted_train_timestamp = sorted((set(train_timestamp))) # unique timestamps in the train set
tscv = TimeSeriesSplit(n_splits=5, test_size=20) # splitting train and validations sets for cross validation
for train_idx, validation_idx in tscv.split(sorted_train_timestamp): # get train and validation sets
    # print("TRAIN:", train_idx, "VALIDATION:", validation_idx)
    train_folds.append([sorted_train_timestamp[i] for i in train_idx]) # folds in train set
    validation_folds.append([sorted_train_timestamp[i] for i in validation_idx]) # folds in validation set

n_epochs = 50
runs_per_fold = 10

cv_ginis = []
val_preds = []
test_preds = []
# network training
for fold in range(len(train_folds)): # iterate cross-validation folds
    fold_idx_train = [ts in train_folds[fold] for ts in train_timestamp]
    fold_idx_val = [ts in validation_folds[fold] for ts in train_timestamp]
    X_train_f, X_val_f = train_X[fold_idx_train,:], train_X[fold_idx_val,:] # X_train for the current fold
    y_train_f, y_val_f = train_y[fold_idx_train,:], train_y[fold_idx_val,:] # y_train for the current fold
    
    for run_n in range(runs_per_fold): # iterate # of runs per fold
        # build the model
        NN, input_dim_NN = build_embedding_network() # build the model and get input dimensions
        
        X_train_f_input = ndarray_to_input_list(X_train_f, input_dim_NN)
        X_val_f_input = ndarray_to_input_list(X_val_f, input_dim_NN)
        X_test_f_input = ndarray_to_input_list(test_X, input_dim_NN)
        
        NN.fit(X_train_f_input, y_train_f, epochs=n_epochs, batch_size=2048, verbose=0)
    
        val_preds.append(NN.predict(X_val_f_input))
        test_preds.append(NN.predict(X_test_f_input))
    
    cv_gini = gini_normalizedc(y_val_f, val_preds) # actual, predicted
    cv_ginis.append(cv_gini)
    for i, cvg in enumerate(cv_gini):
        print('\nFold %d prediction cv gini_%d: %.5f\n' %(fold,i,cv_gini[i]))
    
cv_gini_mean = np.nanmean(np.asarray(cv_ginis),axis=0)
for i, g in enumerate(cv_gini_mean):
    print('Mean validation fold gini_%d: %.5f\n' %(i,g))

test_pred_final = np.nanmean(np.asarray(test_preds),axis=0)


Epoch 1/50
5/5 [==============================] - 1s 5ms/step - loss: -0.8101
Epoch 2/50
5/5 [==============================] - 0s 5ms/step - loss: -1.2111
Epoch 3/50
5/5 [==============================] - 0s 5ms/step - loss: -1.5701
Epoch 4/50
5/5 [==============================] - 0s 5ms/step - loss: -1.8636
Epoch 5/50
5/5 [==============================] - 0s 5ms/step - loss: -2.1885
Epoch 6/50
5/5 [==============================] - 0s 5ms/step - loss: -2.4519
Epoch 7/50
5/5 [==============================] - 0s 5ms/step - loss: -2.6075
Epoch 8/50
5/5 [==============================] - 0s 5ms/step - loss: -2.7030
Epoch 9/50
5/5 [==============================] - 0s 6ms/step - loss: -2.7769
Epoch 10/50
5/5 [==============================] - 0s 6ms/step - loss: -2.8730
Epoch 11/50
5/5 [==============================] - 0s 5ms/step - loss: -2.9160
Epoch 12/50
5/5 [==============================] - 0s 5ms/step - loss: -2.9778
Epoch 13/50
5/5 [==============================] - 0s 5ms/ste

TypeError: only size-1 arrays can be converted to Python scalars